# Project 02: House Price Prediction

**Difficulty**: ⭐ Beginner

**Estimated Time**: 25-30 hours

**Project Type**: Regression

**Dataset**: California Housing Dataset

## Learning Objectives

By the end of this project, you will be able to:
1. Perform regression analysis on real-world data
2. Handle multivariate feature relationships
3. Apply feature selection and regularization techniques
4. Compare multiple regression algorithms
5. Evaluate regression models using appropriate metrics
6. Create predictive models for continuous targets

## Problem Statement

**Goal**: Predict median house prices in California districts based on features like location, demographics, and housing characteristics.

This project demonstrates:
- Regression modeling (vs classification)
- Feature engineering for continuous targets
- Handling geographic and demographic data
- Regularization techniques (Ridge, Lasso, ElasticNet)
- Ensemble methods for regression

## Prerequisites

- Machine Learning Fundamentals (Module 05)
- Linear Regression concepts
- Feature Engineering
- Data Visualization

## 1. Setup and Imports

In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

# Regression Models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

# Metrics
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    mean_absolute_percentage_error
)

# Utilities
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure visualization defaults
%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ All libraries imported successfully!")

## 2. Load and Inspect Data

In [ ]:
# Load California Housing dataset
housing_data = fetch_california_housing(as_frame=True)

# Create DataFrame
df = housing_data.frame

print(f"Dataset shape: {df.shape}")
print(f"Number of rows: {len(df)}")
print(f"Number of columns: {len(df.columns)}")
print("\n" + "="*50)

# Display first few rows
df.head()

In [ ]:
# Display dataset description
print("Dataset Description:")
print(housing_data.DESCR)

In [ ]:
# Display column information
df.info()

In [ ]:
# Statistical summary
df.describe()

### Data Dictionary

| Variable | Definition | Unit |
|----------|------------|------|
| MedInc | Median income in block group | tens of thousands of dollars |
| HouseAge | Median house age in block group | years |
| AveRooms | Average number of rooms per household | rooms |
| AveBedrms | Average number of bedrooms per household | bedrooms |
| Population | Block group population | people |
| AveOccup | Average number of household members | people |
| Latitude | Latitude | degrees |
| Longitude | Longitude | degrees |
| **MedHouseVal** | **Median house value (TARGET)** | **hundreds of thousands of dollars** |

## 3. Exploratory Data Analysis (EDA)

### 3.1 Missing Values Check

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing values by column:")
print(missing_values)
print(f"\nTotal missing values: {missing_values.sum()}")

if missing_values.sum() == 0:
    print("✅ No missing values in this dataset!")

### 3.2 Target Variable Analysis

In [ ]:
# Analyze target variable (MedHouseVal)
target = df['MedHouseVal']

print("House Price Statistics:")
print(f"Mean: ${target.mean():.2f}00k = ${target.mean() * 100:.2f}k")
print(f"Median: ${target.median():.2f}00k = ${target.median() * 100:.2f}k")
print(f"Std Dev: ${target.std():.2f}00k")
print(f"Min: ${target.min():.2f}00k = ${target.min() * 100:.2f}k")
print(f"Max: ${target.max():.2f}00k = ${target.max() * 100:.2f}k")

# Visualize target distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(target, bins=50, color='#4ECDC4', edgecolor='black', alpha=0.7)
axes[0].axvline(target.mean(), color='red', linestyle='--', label=f'Mean: ${target.mean():.2f}00k')
axes[0].axvline(target.median(), color='green', linestyle='--', label=f'Median: ${target.median():.2f}00k')
axes[0].set_xlabel('Median House Value ($100k)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of House Prices')
axes[0].legend()

# Box plot
axes[1].boxplot(target, vert=True)
axes[1].set_ylabel('Median House Value ($100k)')
axes[1].set_title('House Price Box Plot')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Observation: Distribution is right-skewed with a cap at $500k (5.0 in dataset)")

### 3.3 Geographic Visualization

In [ ]:
# Plot house locations colored by price
plt.figure(figsize=(14, 10))
scatter = plt.scatter(df['Longitude'], df['Latitude'],
                     c=df['MedHouseVal'], cmap='viridis',
                     alpha=0.4, s=10)
plt.colorbar(scatter, label='Median House Value ($100k)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('California Housing Prices by Location')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("📊 Key Insight: Coastal areas (especially Bay Area and LA) have highest prices!")

### 3.4 Feature Distributions

In [ ]:
# Plot distributions of all features
fig, axes = plt.subplots(3, 3, figsize=(16, 12))
axes = axes.ravel()

for idx, col in enumerate(df.columns):
    axes[idx].hist(df[col], bins=50, color='#4ECDC4', edgecolor='black', alpha=0.7)
    axes[idx].set_title(col)
    axes[idx].set_ylabel('Frequency')
    axes[idx].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 3.5 Correlation Analysis

In [ ]:
# Compute correlation matrix
correlation_matrix = df.corr()

# Plot heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Show correlations with target
print("\nCorrelations with House Price (MedHouseVal):")
target_corr = correlation_matrix['MedHouseVal'].sort_values(ascending=False)
print(target_corr)

print("\n📊 Key Insights:")
print(f"- Strongest positive correlation: {target_corr.index[1]} ({target_corr.iloc[1]:.3f})")
print(f"- Strongest negative correlation: {target_corr.index[-1]} ({target_corr.iloc[-1]:.3f})")

### 3.6 Relationship: Income vs Price

In [ ]:
# Scatter plot: Median Income vs House Price
plt.figure(figsize=(12, 6))
plt.scatter(df['MedInc'], df['MedHouseVal'], alpha=0.3, s=10, c='#4ECDC4')
plt.xlabel('Median Income ($10k)')
plt.ylabel('Median House Value ($100k)')
plt.title('House Price vs Median Income')
plt.grid(alpha=0.3)

# Add trend line
z = np.polyfit(df['MedInc'], df['MedHouseVal'], 1)
p = np.poly1d(z)
plt.plot(df['MedInc'].sort_values(), p(df['MedInc'].sort_values()),
         "r--", label=f'Trend: y={z[0]:.2f}x+{z[1]:.2f}')
plt.legend()
plt.tight_layout()
plt.show()

print(f"Correlation: {df['MedInc'].corr(df['MedHouseVal']):.3f}")

### 3.7 Outlier Detection

In [ ]:
# Box plots for outlier detection
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.ravel()

for idx, col in enumerate(df.columns[:-1]):  # Exclude target
    axes[idx].boxplot(df[col])
    axes[idx].set_title(col)
    axes[idx].grid(alpha=0.3)

axes[-1].axis('off')  # Hide last empty subplot

plt.tight_layout()
plt.show()

print("📊 Observation: Several features have outliers, especially AveRooms, AveBedrms, and AveOccup")

## 4. Feature Engineering

In [ ]:
# Create a copy for feature engineering
df_engineered = df.copy()

# 1. Rooms per Bedroom (indicator of apartment type)
df_engineered['RoomsPerBedroom'] = df['AveRooms'] / (df['AveBedrms'] + 1e-5)  # Add small value to avoid division by zero

# 2. Population Density (people per household)
df_engineered['PopulationDensity'] = df['Population'] / (df['AveOccup'] + 1e-5)

# 3. Bedroom Ratio (proportion of bedrooms to total rooms)
df_engineered['BedroomRatio'] = df['AveBedrms'] / (df['AveRooms'] + 1e-5)

# 4. Log transformations for skewed features
df_engineered['Log_MedInc'] = np.log1p(df['MedInc'])
df_engineered['Log_AveRooms'] = np.log1p(df['AveRooms'])
df_engineered['Log_Population'] = np.log1p(df['Population'])

# 5. Geographic features (distance from city center approximations)
# Approximate San Francisco coordinates
sf_lat, sf_lon = 37.7749, -122.4194
df_engineered['DistanceFromSF'] = np.sqrt(
    (df['Latitude'] - sf_lat)**2 + (df['Longitude'] - sf_lon)**2
)

# Approximate Los Angeles coordinates
la_lat, la_lon = 34.0522, -118.2437
df_engineered['DistanceFromLA'] = np.sqrt(
    (df['Latitude'] - la_lat)**2 + (df['Longitude'] - la_lon)**2
)

print("✅ New features created:")
new_features = ['RoomsPerBedroom', 'PopulationDensity', 'BedroomRatio',
                'Log_MedInc', 'Log_AveRooms', 'Log_Population',
                'DistanceFromSF', 'DistanceFromLA']
for feat in new_features:
    print(f"  - {feat}")

print(f"\nDataset shape after feature engineering: {df_engineered.shape}")

# Display sample
df_engineered[new_features].head()

## 5. Data Preparation

### 5.1 Feature Selection

In [ ]:
# Select features for modeling
# We'll compare original features vs engineered features

# Original features
original_features = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms',
                    'Population', 'AveOccup', 'Latitude', 'Longitude']

# All features (original + engineered)
all_features = original_features + new_features

# Prepare X and y
X_original = df[original_features]
X_engineered = df_engineered[all_features]
y = df['MedHouseVal']

print(f"Original features: {len(original_features)}")
print(f"Engineered features: {len(all_features)}")
print(f"Target shape: {y.shape}")

### 5.2 Train-Test Split

In [ ]:
# Split data (80/20 split)
X_train_orig, X_test_orig, y_train, y_test = train_test_split(
    X_original, y, test_size=0.2, random_state=42
)

X_train_eng, X_test_eng, _, _ = train_test_split(
    X_engineered, y, test_size=0.2, random_state=42
)

print(f"Training set size: {X_train_orig.shape[0]} ({X_train_orig.shape[0]/len(X_original):.1%})")
print(f"Test set size: {X_test_orig.shape[0]} ({X_test_orig.shape[0]/len(X_original):.1%})")
print(f"\nTarget statistics:")
print(f"Training mean: ${y_train.mean():.2f}00k")
print(f"Test mean: ${y_test.mean():.2f}00k")

### 5.3 Feature Scaling

In [ ]:
# Scale features
scaler_orig = StandardScaler()
X_train_orig_scaled = scaler_orig.fit_transform(X_train_orig)
X_test_orig_scaled = scaler_orig.transform(X_test_orig)

scaler_eng = StandardScaler()
X_train_eng_scaled = scaler_eng.fit_transform(X_train_eng)
X_test_eng_scaled = scaler_eng.transform(X_test_eng)

print("✅ Features scaled successfully!")
print(f"\nScaled training data shape: {X_train_orig_scaled.shape}")
print(f"Scaled test data shape: {X_test_orig_scaled.shape}")

## 6. Model Training and Evaluation

### 6.1 Baseline: Simple Linear Regression

In [ ]:
# Train baseline model
baseline_model = LinearRegression()
baseline_model.fit(X_train_orig, y_train)

# Make predictions
y_pred_baseline = baseline_model.predict(X_test_orig)

# Evaluate
rmse_baseline = np.sqrt(mean_squared_error(y_test, y_pred_baseline))
mae_baseline = mean_absolute_error(y_test, y_pred_baseline)
r2_baseline = r2_score(y_test, y_pred_baseline)

print("Baseline Linear Regression Results:")
print(f"RMSE: ${rmse_baseline:.4f}00k = ${rmse_baseline * 100:.2f}k")
print(f"MAE: ${mae_baseline:.4f}00k = ${mae_baseline * 100:.2f}k")
print(f"R² Score: {r2_baseline:.4f}")

# Visualize predictions
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_baseline, alpha=0.3, s=10, c='#4ECDC4')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
         'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual Price ($100k)')
plt.ylabel('Predicted Price ($100k)')
plt.title(f'Baseline: Actual vs Predicted (R² = {r2_baseline:.3f})')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 6.2 Train Multiple Models

In [ ]:
# Define regression models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(random_state=42),
    'Lasso Regression': Lasso(random_state=42, max_iter=10000),
    'ElasticNet': ElasticNet(random_state=42, max_iter=10000),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42, n_estimators=100),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'SVR': SVR(),
    'KNN': KNeighborsRegressor()
}

# Train and evaluate each model on ORIGINAL features
results_original = {}

print("Training models on ORIGINAL features...\n")
for name, model in models.items():
    # Use scaled data for models that benefit from it
    if name in ['Ridge Regression', 'Lasso Regression', 'ElasticNet', 'SVR', 'KNN']:
        model.fit(X_train_orig_scaled, y_train)
        y_pred = model.predict(X_test_orig_scaled)
    else:
        model.fit(X_train_orig, y_train)
        y_pred = model.predict(X_test_orig)
    
    # Calculate metrics
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    
    results_original[name] = {
        'RMSE': rmse,
        'MAE': mae,
        'R² Score': r2,
        'MAPE (%)': mape,
        'model': model,
        'predictions': y_pred
    }
    
    print(f"✅ {name}: RMSE={rmse:.4f}, R²={r2:.4f}")

print("\n✅ All models trained successfully!")

### 6.3 Compare Model Performance

In [ ]:
# Create comparison dataframe
results_df = pd.DataFrame({
    name: {metric: values[metric] for metric in ['RMSE', 'MAE', 'R² Score', 'MAPE (%)']}
    for name, values in results_original.items()
}).T

# Sort by R² Score
results_df_sorted = results_df.sort_values('R² Score', ascending=False)

print("Model Performance Comparison (Original Features):")
print(results_df_sorted)

# Find best model
best_model_name = results_df['R² Score'].idxmax()
print(f"\n🏆 Best Model: {best_model_name} with R² = {results_df.loc[best_model_name, 'R² Score']:.4f}")

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics = ['RMSE', 'MAE', 'R² Score', 'MAPE (%)']
colors = ['#FF6B6B', '#4ECDC4', '#95E1D3', '#FFD93D']

for idx, (metric, color) in enumerate(zip(metrics, colors)):
    ax = axes[idx // 2, idx % 2]
    
    if metric == 'R² Score':
        sorted_results = results_df.sort_values(metric, ascending=True)
    else:
        sorted_results = results_df.sort_values(metric, ascending=False)  # Lower is better
    
    sorted_results[metric].plot(kind='barh', ax=ax, color=color)
    ax.set_xlabel(metric)
    ax.set_title(f'Model Comparison: {metric}')
    
    # Add value labels
    for i, v in enumerate(sorted_results[metric]):
        ax.text(v + 0.01, i, f'{v:.3f}', va='center')

plt.tight_layout()
plt.show()

### 6.4 Prediction Visualization - Best Model

In [ ]:
# Get best model predictions
best_predictions = results_original[best_model_name]['predictions']
best_r2 = results_original[best_model_name]['R² Score']

# Create visualizations
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter plot: Actual vs Predicted
axes[0].scatter(y_test, best_predictions, alpha=0.3, s=10, c='#4ECDC4')
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
             'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Price ($100k)')
axes[0].set_ylabel('Predicted Price ($100k)')
axes[0].set_title(f'{best_model_name}: Actual vs Predicted (R² = {best_r2:.3f})')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Residuals plot
residuals = y_test - best_predictions
axes[1].scatter(best_predictions, residuals, alpha=0.3, s=10, c='#FF6B6B')
axes[1].axhline(y=0, color='black', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted Price ($100k)')
axes[1].set_ylabel('Residuals')
axes[1].set_title(f'{best_model_name}: Residual Plot')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Residual Statistics:")
print(f"Mean: {residuals.mean():.4f}")
print(f"Std Dev: {residuals.std():.4f}")
print(f"Min: {residuals.min():.4f}")
print(f"Max: {residuals.max():.4f}")

### 6.5 Feature Importance (Random Forest)

In [ ]:
# Get Random Forest model
rf_model = results_original['Random Forest']['model']

# Get feature importances
feature_importance = pd.DataFrame({
    'Feature': original_features,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("Feature Importance (Random Forest):")
print(feature_importance)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Feature'], feature_importance['Importance'], color='#4ECDC4')
plt.xlabel('Importance')
plt.title('Feature Importance - Random Forest')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\n📊 Key Insights:")
top_3 = feature_importance.head(3)['Feature'].tolist()
print(f"Top 3 most important features: {', '.join(top_3)}")

## 7. Hyperparameter Tuning

In [ ]:
# Hyperparameter tuning for Gradient Boosting (one of the best performers)
print("Performing hyperparameter tuning for Gradient Boosting...\n")

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'min_samples_split': [2, 5, 10]
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(
    GradientBoostingRegressor(random_state=42),
    param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_orig, y_train)

print(f"\n✅ Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation R² score: {grid_search.best_score_:.4f}")

# Evaluate on test set
y_pred_tuned = grid_search.predict(X_test_orig)
rmse_tuned = np.sqrt(mean_squared_error(y_test, y_pred_tuned))
r2_tuned = r2_score(y_test, y_pred_tuned)

print(f"\nTest set R² (before tuning): {results_original['Gradient Boosting']['R² Score']:.4f}")
print(f"Test set R² (after tuning): {r2_tuned:.4f}")
print(f"Test set RMSE (after tuning): ${rmse_tuned:.4f}00k = ${rmse_tuned * 100:.2f}k")
print(f"Improvement: {(r2_tuned - results_original['Gradient Boosting']['R² Score']):.4f}")

## 8. Cross-Validation Analysis

In [ ]:
# Perform cross-validation on top models
print("Performing 5-fold cross-validation...\n")

cv_results = {}
top_models = ['Random Forest', 'Gradient Boosting', 'Ridge Regression']

for name in top_models:
    model = results_original[name]['model']
    
    # Use scaled data for Ridge
    if name == 'Ridge Regression':
        scores = cross_val_score(model, X_train_orig_scaled, y_train,
                                cv=5, scoring='r2')
    else:
        scores = cross_val_score(model, X_train_orig, y_train,
                                cv=5, scoring='r2')
    
    cv_results[name] = {
        'Mean': scores.mean(),
        'Std': scores.std(),
        'Min': scores.min(),
        'Max': scores.max()
    }
    
    print(f"{name}:")
    print(f"  Mean R²: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")
    print(f"  Range: [{scores.min():.4f}, {scores.max():.4f}]\n")

# Visualize CV results
cv_df = pd.DataFrame(cv_results).T
cv_df = cv_df.sort_values('Mean', ascending=True)

plt.figure(figsize=(10, 6))
plt.barh(cv_df.index, cv_df['Mean'], xerr=cv_df['Std'], color='#4ECDC4', alpha=0.7)
plt.xlabel('Mean Cross-Validation R² Score')
plt.title('5-Fold Cross-Validation Results')

# Add value labels
for i, (idx, row) in enumerate(cv_df.iterrows()):
    plt.text(row['Mean'] + 0.01, i, f"{row['Mean']:.3f}", va='center')

plt.tight_layout()
plt.show()

## 9. Key Insights and Conclusions

### Summary of Findings

**1. Price Patterns:**
- Average house price: ~$200k (in 1990s dollars)
- Strong geographic influence (coastal areas more expensive)
- Median income is the strongest predictor (correlation: 0.69)
- Location (Latitude/Longitude) also highly important

**2. Most Important Features:**
- Median Income (MedInc) - Strongest predictor
- Location (Latitude, Longitude) - Geographic premium
- House Age - Moderate importance
- Average Rooms/Bedrooms - Housing quality indicators

**3. Model Performance:**
- Best models: Random Forest and Gradient Boosting
- Achieved R² scores of ~0.80-0.82
- Prediction error (RMSE): ~$50-55k
- Linear models (Ridge/Lasso) performed reasonably well (R² ~0.60)
- Complex models significantly outperformed simple linear regression

**4. Business Insights:**
- Income is the primary driver of house prices (socioeconomic factor)
- Location matters significantly (coastal premium)
- Older houses don't necessarily have lower prices (vintage value)
- Average occupancy has minimal impact on price

**5. Model Limitations:**
- Capped target at $500k may affect predictions for luxury homes
- Block-level data averages out individual house characteristics
- No information about house condition, school quality, crime rates
- Data is from 1990s (prices adjusted for modern use)

## 10. Next Steps and Improvements

**Potential Improvements:**
1. **Feature Engineering:**
   - Test engineered features (RoomsPerBedroom, DistanceFromSF, etc.)
   - Create interaction features (Income × Location)
   - Polynomial features for non-linear relationships

2. **Advanced Modeling:**
   - XGBoost and LightGBM
   - Neural networks
   - Ensemble methods (stacking)

3. **Error Analysis:**
   - Analyze predictions with large errors
   - Identify patterns in misclassifications
   - Address outliers and capped values

4. **Model Deployment:**
   - Create Streamlit app for price predictions
   - Deploy as REST API with FastAPI
   - Dockerize for production

5. **Further Analysis:**
   - Geographic clustering analysis
   - Time series analysis if temporal data available
   - Compare with real estate APIs (Zillow, Redfin)

## Exercises

Try these exercises to deepen your understanding:

### Exercise 1: Test Engineered Features
Train the models using the engineered features (X_engineered). Do they improve R² scores?

### Exercise 2: Polynomial Features
Use `PolynomialFeatures` to create quadratic features. How does this affect linear regression performance?

### Exercise 3: Learning Curves
Plot learning curves to diagnose bias vs variance. Are the models overfitting or underfitting?

### Exercise 4: Feature Selection
Use Lasso's coefficients to identify and remove unimportant features. Does this simplify the model without hurting performance?

### Exercise 5: Deployment
Create a Streamlit app that takes house characteristics as input and predicts price with the best model.

## Project Checklist

✅ **Completed:**
- [x] Data loading and exploration
- [x] Comprehensive EDA with visualizations
- [x] Geographic analysis
- [x] Correlation analysis
- [x] Feature engineering
- [x] Train-test split
- [x] Feature scaling
- [x] Multiple regression models (9 algorithms)
- [x] Model comparison with multiple metrics
- [x] Hyperparameter tuning
- [x] Cross-validation analysis
- [x] Feature importance analysis
- [x] Residual analysis
- [x] Clear insights and conclusions

📋 **For Portfolio:**
- [ ] Create professional README.md
- [ ] Add requirements.txt
- [ ] Test engineered features
- [ ] Create presentation slides
- [ ] Deploy as web app (optional)
- [ ] Write blog post (optional)